<h2>Capstone  Project</h2>
By Mónica Atiaga

Batch - DSNB1222

<h1>SHOE BRAND CAPSTONE</h1>


### Phase 1                                                                                                                                            
#### Web scrapping the below given six URL :
(21 marks)
- https://www.nike.com
- https://www.adidas.com
- https://us.puma.com/us/en
- https://www.skechers.com
- https://www.columbia.com/
- https://www.woodlandworldwide.com/

Make the table in  csv format  (3 csv files)  
1. Table1.csv  
    - ShoeName ,Category(Men/Women), no.of colors,price  
2. Table2.csv
    - count of sizes, colors 1, color 2, color3 ,color4, color5, Style code/product code  
3. Table3.csv 
    - reviews, Size, comfort, durability /quality/performance(quantification), star rating,

In [1]:
# Importing libraries 
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, ElementClickInterceptedException


from bs4 import BeautifulSoup
import time
import regex as re


#### My Custom Functions (at the end section)


* get_attribute_el(webelement, attribute)
* text_el(webelement)
* go_next_page(driver, xpath)
* load_items_by(driver, n=None, **kwargs)
* remove_popup(xpath)
* scrape_item_detail(driver, not_found_value = '-', **kwargs)
* scrape_data_by_urls(urls, attributes_det)
* web_scraping_table(xpath_table, init_data=1, footer=False)

All functions used are my own. As we have progressed in the assignments, I have been improving the code so that they are more general and can be applied in different scenarios. I decided to do it this way to improve and progress my knowledge of python.
By Monica Atiaga 

In [2]:
# Define the sites to scrap
sites = {
'nike': 'https://www.nike.com',
'adidas': 'https://www.adidas.com',
'puma': 'https://us.puma.com/us/en',
'skechers': 'https://www.skechers.com',
'columbia': 'https://www.columbia.com/',
'woodlww': 'https://www.woodlandworldwide.com/',
}

## Nike.com

In [168]:
# Open the Driver 
driver = webdriver.Chrome()

url = sites['nike']
driver.maximize_window()


driver.get(url) 

# Going to United State Market
path_nike = {'msg': ['United State market...'],
             'xpaths': ['/html/body/div[5]/div/div/nav/div/div/div[2]/div/a[9]/div/div[2]/p']}

follow_path_to(path_nike) 

United State market...


In [169]:
%%time

# Initialization of the dataframe
df_shoes = pd.DataFrame()

# Scrap the main data for Men and Women in nike.com
for category in ['Men','Women']:

    path_nike = {'msg': [ f'\n{category} Category...', 'Looking for Shoes...\n'],
                 'xpaths': [f'//a[@aria-label="{category}"]', '//a[@aria-label="Shoes"]']}

    follow_path_to(path_nike)  

    # Find out the number of shoes to retrieve in the category
    pattern = r'\((\d+)\)' # Remove the parentheses
    n_shoes = n_shoes_to_scrap('//span[@class="wall-header__item_count"]', pattern)  
    print(f"Shoes to recover: {n_shoes}\n" )

    # Define the xpath for shoes card 
    xpath = '//div[@class="product-card product-grid__card  css-1t0asop"]'

    # Scroll until recovered all shoes
    recovered_shoes, _ = scroll_to_recovered_shoes(n_shoes, xpath)

    print(f"\nTotal of shoes recovered: {recovered_shoes}\n")

    print('*'*50)
    print(' '*5, f'S C R A P I N G - Category: {category}')
    print('*'*50)

    # Scraping ShoeName ,Category(Men/Women), no.of colors,price
    attributes = {'ShoeName': ['//div[@class="product-card__title"]'],
                  'URL': ['//a[@data-testid="product-card__link-overlay"]', fn_href],
                  'no_colors': ['//div[@class="product-card__product-count"]'],
                  'price_reduce': ['//div[@data-testid="product-card__price"]'],
                  'price': ['//div[@data-testid="product-price"]']}

    df_shoes_cat = load_items_by_xpath(driver, None, **attributes) 

    # Setting the category
    df_shoes_cat['Category'] = category

    # Append the new data 
    df_shoes = pd.concat([df_shoes, df_shoes_cat])
    
    print( f'Products in category {category} retrieved successfully: {len(df_shoes)}' )



Men Category...
Looking for Shoes...

Shoes to recover: 985

Recovered shoes: 24
Scrolling 1 times...
Recovered shoes: 72, still to recovered: 913
Scrolling 2 times...
Recovered shoes: 120, still to recovered: 865
Scrolling 3 times...
Recovered shoes: 168, still to recovered: 817
Scrolling 4 times...
Recovered shoes: 216, still to recovered: 769
Scrolling 5 times...
Recovered shoes: 264, still to recovered: 721
Scrolling 6 times...
Recovered shoes: 312, still to recovered: 673
Scrolling 7 times...
Recovered shoes: 360, still to recovered: 625
Scrolling 8 times...
Recovered shoes: 408, still to recovered: 577
Scrolling 9 times...
Recovered shoes: 456, still to recovered: 529
Scrolling 10 times...
Recovered shoes: 504, still to recovered: 481
Scrolling 11 times...
Recovered shoes: 552, still to recovered: 433
Scrolling 12 times...
Recovered shoes: 600, still to recovered: 385
Scrolling 13 times...
Recovered shoes: 648, still to recovered: 337
Scrolling 14 times...
Recovered shoes: 696, 

In [171]:
# Cleaning the data frame
df_shoes['no_colors'] = df_shoes['no_colors'].str.extract(r'(\d+)')

df_shoes['price_reduce'] = df_shoes['price_reduce'].str.extract(r'\$(\d+(\.\d+)?)')[0]
# df_shoes['price_reduce'] = df_shoes['price_reduce'].astype(float)

df_shoes['price'] = df_shoes['price'].str.extract(r'\$(\d+(\.\d+)?)')[0]
# df_shoes['price'] = df_shoes['price'].astype(float)

# Setting up the dataframe
df_shoes['Brand'] = 'Nike'
df_shoes.reset_index(drop=True, inplace=True)

In [173]:
# Saving the data frame
df_shoes.to_csv('datasets/Table1_nike.csv')
display(df_shoes)

,ShoeName,URL,no_colors,price_reduce,price,Category
0,Nike Dunk Low,https://www.nike.com/t/dunk-low-shoes-rv2vSN/F...,1,115,115,Men
1,"Air Jordan 1 High OG ""Royal Reimagined""",https://www.nike.com/t/air-jordan-1-high-og-ro...,1,180,180,Men
2,Nike Dunk Low Retro,https://www.nike.com/t/dunk-low-retro-mens-sho...,1,115,115,Men
3,Nike Dunk High Premium,https://www.nike.com/t/dunk-high-premium-mens-...,1,135,135,Men
4,Nike Vomero 5,https://www.nike.com/t/vomero-5-mens-shoes-Mgs...,3,160,160,Men
...,...,...,...,...,...,...
1627,Nike Superfly 9 Elite Mercurial Dream Speed,https://www.nike.com/t/superfly-9-elite-mercur...,2,250.97,295,Women
1628,Nike React Gato,https://www.nike.com/t/react-gato-indoor-court...,3,140,140,Women
1629,Nike Superfly 9 Academy Mercurial Dream Speed,https://www.nike.com/t/superfly-9-academy-merc...,3,82.97,110,Women
1630,Nike Tiempo Legend 10 Academy,https://www.nike.com/t/tiempo-legend-10-academ...,4,80,80,Women


In [ ]:
# //div[@class="mt4-sm mb2-sm mr0-lg ml0-lg mt5-sm mb3-sm body-2 css-1j3x2vp"]/div
# //div[@class="colorway-images-wrapper"]/fieldset/div[1]/div/label/img
#//li[@class="description-preview__style-color ncss-li"] #Style code

In [ ]:
# count of sizes, colors 1, color 2, color3 ,color4, color5, Style code/product code

attributes_det = {'count_of_sizes': [By.XPATH, '//p[@class="f4 my-3"]', fn_text, False],
                  'style_code' : [By.XPATH, '//li[@class="description-preview__style-color ncss-li"]', fn_text, False],
                  'Language used': [By.XPATH, '//span[@class="color-fg-default text-bold mr-1"]', fn_text, True]}


In [189]:
df_shoes[df_shoes['no_colors'].astype(int) > 5]['URL'][:5]

6     https://www.nike.com/t/kd16-basketball-shoes-H...
7     https://www.nike.com/t/vaporfly-3-mens-road-ra...
18    https://www.nike.com/t/air-max-270-mens-shoes-...
28    https://www.nike.com/t/air-jordan-1-zoom-cmft-...
35    https://www.nike.com/t/lebron-nxxt-gen-basketb...
Name: URL, dtype: object

In [ ]:
# Scraping detail info
wait_time = 5
time.sleep(wait_time)

# df_det = scrape_data_by_urls(df_shoes['URL'][:5], attributes_det)

df_det = scrape_data_by_urls(df_shoes[df_shoes['no_colors'].astype(int) > 5]['URL'][:5],
                             attributes_det)


# nike details

In [17]:
df_shoes_nike = pd.read_csv('datasets/Table1_nike.csv', index_col=0)
df_shoes_nike.head(10)

,ShoeName,URL,no_colors,price_reduce,price,Category
0,Nike Dunk Low,https://www.nike.com/t/dunk-low-shoes-rv2vSN/F...,1,115.00,115,Men
1,"Air Jordan 1 High OG ""Royal Reimagined""",https://www.nike.com/t/air-jordan-1-high-og-ro...,1,180.00,180,Men
2,Nike Dunk Low Retro,https://www.nike.com/t/dunk-low-retro-mens-sho...,1,115.00,115,Men
3,Nike Dunk High Premium,https://www.nike.com/t/dunk-high-premium-mens-...,1,135.00,135,Men
4,Nike Vomero 5,https://www.nike.com/t/vomero-5-mens-shoes-Mgs...,3,160.00,160,Men
5,Tiger Woods '13,https://www.nike.com/t/tiger-woods-13-mens-gol...,3,250.00,250,Men
6,KD16,https://www.nike.com/t/kd16-basketball-shoes-H...,6,160.00,160,Men
7,Nike Vaporfly 3,https://www.nike.com/t/vaporfly-3-mens-road-ra...,7,260.00,260,Men
8,Nike Air Max 2 CB '94,https://www.nike.com/t/air-max-2-cb-94-mens-sh...,1,144.97,170,Men
9,Nike Air Flight Huarache,https://www.nike.com/t/air-flight-huarache-men...,3,106.97,125,Men


In [34]:
driver.quit()

In [35]:
%%time
# Open the Driver 
driver = webdriver.Chrome()
driver.maximize_window()

# Initializing the dataframe
df_det = pd.DataFrame()
total_urls = 100

# for i, url in enumerate(df_shoes_puma['URL'][:10]):
url = df_shoes_nike['URL'][0]
driver.get(url) 

# Going to United State Market


# path_nike = {'msg': ['United State market...'],
#              'xpaths': ['//a[@data-country="us"]']}
# #              'xpaths': ['//p[@class="nav-bold" and contains(text(), "United States")]']}

# follow_path_to(path_nike) 

# print(f"Retrieving Details of record: {i+1}/{total_urls}")

Wall time: 1min 22s


In [33]:
# Removing the cookies pop up
popup_xpath = '//button[@aria-label="Close Menu"]'
remove_popup(popup_xpath)

NameError: name 'ElementNotInteractableException' is not defined

In [23]:
url

'https://www.nike.com/t/dunk-low-shoes-rv2vSN/FQ8826-100'

In [22]:
soup_config = {
            'count_of_sizes':['span',{'data-content':'size-value'}, None ,'-'], 
            'colors': ['label', {'data-test-id': 'color'}, None, '-'],
            'product_code': ['li', {}, lambda text: text and 'Style:' in text , '-'],
            'reviews': ['div', {'id' : "product-reviews"}, None, '-'],
            'quality':['p',{'data-test-id':'percentRecommendThisProduct'}, None ,'-'], 
        }

# Getting the HTML code
html_string = driver.page_source   
print(f"Get the HTML code: {url}\n")  

row = bs_scrape_item_detail(html_string, **soup_config)
row

Get the HTML code: https://www.nike.com/t/dunk-low-shoes-rv2vSN/FQ8826-100



{'count_of_sizes': [],
 'colors': [],
 'product_code': ['Style: FQ8826-100'],
 'reviews': [],
 'quality': []}

In [96]:
%%time
# Open the Driver 
driver = webdriver.Chrome()
driver.maximize_window()

# Initializing the dataframe
df_det = pd.DataFrame()
total_urls = 100

for i, url in enumerate(df_shoes_puma['URL'][:10]):
#     url = df_shoes_puma['URL'][10]
    driver.get(url) 
    
    print(f"Retrieving Details of record: {i+1}/{total_urls}")

    if i == 0:

        wait_time = 3
        time.sleep(wait_time)

        # Removing the cookies pop up
        popup_xpath = '//div[@data-test-id="cookie-banner"]/button'
        remove_popup(popup_xpath)

    soup_config = {
            'count_of_sizes':['span',{'data-content':'size-value'}, None ,'-'], 
            'colors': ['label', {'data-test-id': 'color'}, None, '-'],
            'product_code': ['li', {}, lambda text: text and 'Style:' in text , '-'],
            'reviews': ['div', {'id' : "product-reviews"}, None, '-'],
            'quality':['p',{'data-test-id':'percentRecommendThisProduct'}, None ,'-'], 
        }

    # Break the scroll into smaller steps 
    for _ in range(5):
        driver.execute_script("window.scrollBy(0, 400);")

    wait_time = 3
    time.sleep(wait_time)

    # Show Reviews
    xpath_link = '//button[@data-test-id="read-all-reviews"]'
    try:
        link = driver.find_element(By.XPATH, xpath_link)
        link.click()
        print(f'Show reviews of {i}')
    except NoSuchElementException as e:
        print(f"Item {df_shoes_puma.iloc[i]['ShoeName']} does not have reviews.")

    time.sleep(wait_time)

    # Getting the HTML code
    html_string = driver.page_source   
    print(f"Get the HTML code: {url}\n")  

    row = bs_scrape_item_detail(html_string, **soup_config)

    df_det = df_det.append(row, ignore_index=True)

# Free driver
driver.quit()

Retrieving Details of record: 1/100
Pop-up removed
Show reviews of 0
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 2/100
Show reviews of 1
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=02&referrer-category=womens-shop-all-womens

Retrieving Details of record: 3/100
Show reviews of 2
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=03&referrer-category=womens-shop-all-womens

Retrieving Details of record: 4/100
Item Basket Classic Velvet Women's SneakersMalachite-Puma Team Gold-PUMA White does not have reviews.
Get the HTML code: https://us.puma.com/us/en/pd/basket-classic-velvet-womens-sneakers/398949?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 5/100
Item Basket Classic Velvet Women's SneakersDark Jas

In [138]:
# Backup dataframe detail brand Puma
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_puma.csv')

## puma

In [36]:
sites['puma']

'https://us.puma.com/us/en'

In [39]:
driver.quit()

In [93]:
# Open the Driver 
driver = webdriver.Chrome()
url = sites['puma']
driver.get(url) 

driver.maximize_window()

wait_time = 3
time.sleep(wait_time)

# Removing the cookies pop up
popup_xpath = '//div[@data-test-id="cookie-banner"]/button'
remove_popup(popup_xpath)

Pop-up removed


In [43]:
%%time
# Scrap the main data for Men and Women in nike.com
# Initialization of the dataframe
df_shoes_puma = pd.DataFrame()
categories = {'Women': 'Footwear', 'Men': 'Shoes'}

for category, value in categories.items():
    
    # Follows the path to the category specified
    path_puma = {'msg': [f'\n{category} section...\n',
                         'Get 20% off clicked...\n',
                         'Category clicked...\n', 
                         'Shoes selected...\n', 
                         'Closing category menu...\n'],
                 'xpaths': [f'//a[@data-link-name="{category}"]',
                            '//button[@data-test-id="newsletter-sign-up-form-button"]/div',
                            '//button[@aria-controls="productdivName-filter-dropdown"]',
                            f'//label[@for="{value}"]',
                            '//button[@data-test-id="close-btn"]'
                            ],
                'wait_time': [ 5, 5, 8, 4, 4, 1]}

    follow_path_to(path_puma) 
    
    wait_time = 5
    time.sleep(wait_time)

    # Find out the number of shoes to retrieve in the category
    n_shoes = n_shoes_to_scrap('//span[@data-test-id="product-results"]')
    print(f"Shoes to recover: {n_shoes}\n" )

    # Define the xpath for shoes card 
    xpath = '//li[@data-test-id="product-list-item"]'

    # Scroll until recovered all shoes
    recovered_shoes, _ = scroll_to_recovered_shoes(n_shoes, xpath)
    print(f"\nTotal of shoes recovered: {recovered_shoes}\n")

    print('*'*50)
    print(' '*5, f'S C R A P I N G - Category: {category}')
    print('*'*50)

    # Scraping ShoeName ,Category(Men/Women), no.of colors, price
    xpath_grp = '//li[@data-test-id="product-list-item"]'
    soup_config = {
        'ShoeName':['h3',{}, None ,'-'], 
        'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', '-'],
        'no_colors': ['div', {'class':"relative h-12 mobile:h-10 flex items-center group"}, None, '-'],
        'price_reduce': ['span', {'data-test-id': 'sale-price'}, None, '-' ],
        'price': ['span', {'data-test-id':"price"}, None, '-'],
    }
    # time.sleep(wait_time)
    df_shoes_cat = scrape_items_by_group(driver, xpath_grp, **soup_config)

    # Setting the category
    df_shoes_cat['Category'] = category

    # Append the new data 
    df_shoes_puma = pd.concat([df_shoes_puma, df_shoes_cat])

print( f'Products in category {category} retrieved successfully: {len(df_shoes_puma)}' )

Wait time ** 5 **

Women section...

Wait time ** 5 **
The link can not be found: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-test-id="newsletter-sign-up-form-button"]/div"}
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Wait time ** 8 **
Category clicked...

Wait time ** 4 **
Shoes selected...

Wait time ** 4 **
Closing category menu...

Shoes to recover: 249

Recovered shoes: 24
Scrolling 1 times...
Recovered shoes: 48, still to recovered: 201
Scrolling 2 times...
Recovered shoes: 72, still to recovered: 177
Scrolling 3 times...
Recovered shoes: 96, still to recovered: 153
Scrolling 4 times...
Recovered shoes: 120, still to recovered: 129
Scrolling 5 times...
Recovered shoes: 144, still to recovered: 105
Scrolling 6 times...
Recovered shoes: 168, still to recovered: 81
Scrolling 7 times...
Recovered shoes

In [45]:
# Free Driver
driver.quit()

# Cleaning the data frame
df_shoes_puma['no_colors'] = df_shoes_puma['no_colors'].str.extract(r'(\d+)')
df_shoes_puma['price_reduce'] = df_shoes_puma['price'].str.extract(r'\$(\d+(\.\d+)?)')[0]
df_shoes_puma['price'] = df_shoes_puma['price'].str.extract(r'\$(\d+(\.\d+)?)')[0]
df_shoes_puma['URL'] = 'https://us.puma.com'+ df_shoes_puma['URL']

# Setting up the dataframe
df_shoes_puma['Brand'] = 'Puma'
df_shoes_puma.reset_index(drop=True, inplace=True)

In [46]:
# Saving the data frame
df_shoes_puma.to_csv('datasets/Table1_puma.csv')
display(df_shoes_puma)

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
0,FENTY x PUMA Creeper Phatty Women's SneakersPU...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
1,FENTY x PUMA Creeper Phatty Women's SneakersLa...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
2,FENTY x PUMA Creeper Phatty Women's SneakersSp...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
3,Basket Classic Velvet Women's SneakersMalachit...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.00,80.00,Women,Puma
4,Basket Classic Velvet Women's SneakersDark Jas...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.00,80.00,Women,Puma
...,...,...,...,...,...,...,...
651,Suede Mid XXI SneakersPuma Black-Puma White-Am...,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma
652,Speedcat OG + Sparco Driving ShoesPeacoat-Puma...,https://us.puma.com/us/en/pd/speedcat-og-%2B-s...,5,100.00,100.00,Men,Puma
653,Suede Mid XXI SneakersQuarry-Puma White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma
654,Suede Mid XXI SneakersPumpkin Pie-PUMA White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma


In [50]:
df_shoes_puma = pd.read_csv('datasets/Table1_puma.csv', index_col=0)
df_shoes_puma.head()

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
0,FENTY x PUMA Creeper Phatty Women's SneakersPU...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma
1,FENTY x PUMA Creeper Phatty Women's SneakersLa...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma
2,FENTY x PUMA Creeper Phatty Women's SneakersSp...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma
3,Basket Classic Velvet Women's SneakersMalachit...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.0,80.0,Women,Puma
4,Basket Classic Velvet Women's SneakersDark Jas...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.0,80.0,Women,Puma


In [99]:
driver.quit()

In [96]:
def scrape_details_by_URL_in_batches(df_urls, soup_config, **kwargs):

    # Open the Driver 
    driver = webdriver.Chrome()
    driver.maximize_window()

    # Initializing the dataframe
    df_det = pd.DataFrame()
    total_urls = df_urls.shape[0]
    url_error = pd.DataFrame(columns=['index','URL','Exception','msg' ])

    for i, url in enumerate(df_urls['URL']):
        driver.get(url) 

        print(f"Retrieving Details of record: {i+1}/{total_urls}")

        wait_time = 3
        time.sleep(wait_time)
        
        # Removing cookie pop-up if first iteration and specified
        if 'xpath_remove_popup' in kwargs and i == 0:
            remove_popup(kwargs['xpath_remove_popup'])

        # Break the scroll into smaller steps 
        if 'n_scrolls' in kwargs: 
            for _ in range(kwargs['n_scrolls']):
                height = kwargs['height_scroll'] if 'height_scroll' in kwargs else 400
                driver.execute_script(f"window.scrollBy(0, {height});")

        wait_time = 3
        time.sleep(wait_time)

        # Show Reviews
        if 'xpath_reviews' in kwargs:
            xpath_link = kwargs['xpath_reviews']
            try:
                link = driver.find_element(By.XPATH, xpath_link)
                link.click()
                print(f'Show reviews of {i + 1}')
            except NoSuchElementException as e:
                print(f"Item {df_urls.iloc[i]['ShoeName']} has not reviews.")
            except Exception as e:
                print(f"Review Exception of {df_urls.index[i]}")
                row_error = {'index': df_urls.index[i],
                             'URL': url,
                             'Exception': type(e),
                             'msg': e.msg,} 
                url_error = url_error.append(row_error, ignore_index=True)
            time.sleep(wait_time)

        # Getting the HTML code
        html_string = driver.page_source   
        print(f"Get the HTML code: {url}\n")  

        # Scrape details with Beautiful Soup
        row = bs_scrape_item_detail(html_string, **soup_config)
        df_det = df_det.append(row, ignore_index=True)

    # Free driver
#     driver.quit()
    
    return [df_det, url_error]

In [98]:
%%time
# Define parameters
df_urls = df_shoes_puma[['ShoeName', 'URL']][:10]
soup_config = {
            'count_of_sizes':['span',{'data-content':'size-value'}, None ,'-'], 
            'colors': ['label', {'data-test-id': 'color'}, None, '-'],
            'product_code': ['li', {}, lambda text: text and 'Style:' in text , '-'],
            'reviews': ['div', {'id' : "product-reviews"}, None, '-'],
            'quality':['p',{'data-test-id':'percentRecommendThisProduct'}, None ,'-'], 
        }

config = {
    'xpath_remove_popup': '//div[@data-test-id="cookie-banner"]/button',
    'xpath_show_reviews': '//button[@data-test-id="read-all-reviews"]',
    'n_scrolls': 4,
    'height_scroll': 400,
}

df_def, url_error = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/10


NameError: name 'ElementNotInteractableException' is not defined

In [91]:
urls_error

['https://us.puma.com/us/en/pd/rs-x-soft-womens-sneakers/393772?swatch=02&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/puma-180-prm-womens-sneakers/393764?swatch=02&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/bmw-m-motorsport-rs-puls-womens-sneakers/307761?swatch=01&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/seasons-fast-trac-nitro%E2%84%A2-2-womens-running-shoes/307685?swatch=03&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/seasons-voyage-nitro%E2%84%A2-3-womens-running-shoes/377746?swatch=07&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/seasons-voyage-nitro%E2%84%A2-2-womens-running-shoes/376946?swatch=08&referrer-category=womens-shop-all-womens',
 'https://us.puma.com/us/en/pd/cali-dream-preppy-womens-sneakers/389876?swatch=02&referrer-category=womens-shop-all-womens']

In [90]:
len(urls_error)

7

In [59]:
# Backup dataframe detail brand Puma
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_raw_puma.csv')

In [169]:
# Cleaning data frame
df_det['count_of_sizes'] = df_det['count_of_sizes'].apply(len)
temp = df_det_puma_backup['product_code'].dropna().apply(lambda x: re.findall(r'Style: (\d+_\d+)', str(x)))
df_det['product_code'] = temp.apply(lambda x: x[0] if x else None)

df_det['reviews_data'] = df_det['reviews'].dropna().apply(lambda x: re.findall(r'Reviews\D*(\d+)\D*Average:\D*(\d+\.\d+)', str(x)))
df_det['reviews_data'] = pd.DataFrame(df_det['reviews_data'].to_list(), index=df_det.index)
df_det[['n_reviews', 'star_rating']] = pd.DataFrame(df_det['reviews_data'].to_list(), index=df_det.index)

temp = df_det['quality'].dropna().apply(lambda x: re.findall(r'(\d+\.\d+)% recommend this product', str(x)))
df_det['quality'] = temp.apply(lambda x: x[0] if x else None)

# Drop columns no longer needed
df_det.drop(['reviews','reviews_data'], axis=1, inplace=True)

df_det

,count_of_sizes,colors,product_code,quality,n_reviews,star_rating
0,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_01,40.0,5,4.4
1,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_02,40.0,5,4.4
2,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_03,40.0,5,4.4
3,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas...",398949_01,None,0,0.0
4,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas...",398949_02,None,0,0.0
5,0,[],None,None,None,None
6,19,"[Malachite-Warm White, PUMA Black-PUMA Black]",397112_02,None,0,0.0
7,19,"[Malachite-Warm White, PUMA Black-PUMA Black]",397112_01,None,0,0.0
8,11,"[Puma White-Puma Black, Puma White-Peyote]",381983_01,85.7,55,4.4
9,12,"[Puma White-Puma White, Puma White-Puma Black]",369155_04,None,160,4.1


In [181]:
# Setting tables
df_table2_puma = df_det[['product_code', 'count_of_sizes', 'colors']]
df_table3_puma = df_det[['product_code', 'n_reviews', 'quality', 'star_rating']]
df_table3_puma['size'] = np.NaN
df_table3_puma['comfort'] = np.NaN

# Saving the data frame
df_table2_puma.to_csv('datasets/Table2_puma.csv')
display(df_table2_puma)

df_table3_puma.to_csv('datasets/Table3_puma.csv')
display(df_table3_puma)


,product_code,count_of_sizes,colors
0,399332_01,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
1,399332_02,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
2,399332_03,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
3,398949_01,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas..."
4,398949_02,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas..."
5,None,0,[]
6,397112_02,19,"[Malachite-Warm White, PUMA Black-PUMA Black]"
7,397112_01,19,"[Malachite-Warm White, PUMA Black-PUMA Black]"
8,381983_01,11,"[Puma White-Puma Black, Puma White-Peyote]"
9,369155_04,12,"[Puma White-Puma White, Puma White-Puma Black]"


,product_code,n_reviews,quality,star_rating,size,comfort
0,399332_01,5,40.0,4.4,NaN,NaN
1,399332_02,5,40.0,4.4,NaN,NaN
2,399332_03,5,40.0,4.4,NaN,NaN
3,398949_01,0,None,0.0,NaN,NaN
4,398949_02,0,None,0.0,NaN,NaN
5,None,None,None,None,NaN,NaN
6,397112_02,0,None,0.0,NaN,NaN
7,397112_01,0,None,0.0,NaN,NaN
8,381983_01,55,85.7,4.4,NaN,NaN
9,369155_04,160,None,4.1,NaN,NaN


In [164]:
# Establishing 
df_shoes_puma.loc[:9, 'product_code'] = df_det['product_code']
df_shoes_puma[:10]

,Unnamed: 0,ShoeName,URL,no_colors,price_reduce,price,Category,Brand,product_code
0,0,FENTY x PUMA Creeper Phatty Women's SneakersPU...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma,399332_01
1,1,FENTY x PUMA Creeper Phatty Women's SneakersSp...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma,399332_02
2,2,FENTY x PUMA Creeper Phatty Women's SneakersLa...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.0,140.0,Women,Puma,399332_03
3,3,Basket Classic Velvet Women's SneakersMalachit...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.0,80.0,Women,Puma,398949_01
4,4,Basket Classic Velvet Women's SneakersDark Jas...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,80.0,80.0,Women,Puma,398949_02
5,5,Mayze Heritage Velvet Women's SneakersPUMA Whi...,https://us.puma.com/us/en/pd/mayze-heritage-ve...,1,100.0,100.0,Women,Puma,None
6,6,CA Pro Velour SneakersPUMA Black-PUMA Black,https://us.puma.com/us/en/pd/ca-pro-velour-sne...,2,100.0,100.0,Women,Puma,397112_02
7,7,CA Pro Velour SneakersMalachite-Warm White,https://us.puma.com/us/en/pd/ca-pro-velour-sne...,2,100.0,100.0,Women,Puma,397112_01
8,8,Mayze Women's SneakersPuma White-Puma Black,https://us.puma.com/us/en/pd/mayze-womens-snea...,2,90.0,90.0,Women,Puma,381983_01
9,9,Cali Women's SneakersPuma White-Puma Black,https://us.puma.com/us/en/pd/cali-womens-sneak...,2,80.0,80.0,Women,Puma,369155_04


In [25]:
xpath_link = '//button[@data-test-id="read-all-reviews"]'
try:
    link = driver.find_element(By.XPATH, xpath_link)
    link.click()
    print('Show reviews...')
except NoSuchElementException as e:
    print(f"The link can not be found: {e.msg}")

Show reviews...


//section[@aria-label="Product Info"]/div

- count of sizes, colors 1, color 2, color3 ,color4, color5, Style code/product code  

count of sizes 
//span[@data-content="size-value"]

colors: 
//label[@data-test-id="color"]/span

Style: 399332_01
//ul[@data-uds-child="text"]/li

- reviews, Size, comfort, durability /quality/performance(quantification), star rating,

reviews 
//div[@id="product-reviews"]//h2

link reviews:
//button[@data-test-id="read-all-reviews"]

durability/quality/performance(quantification)
//p[@data-test-id="percentRecommendThisProduct"]
5*82.8/100
//button[@data-test-id="close-btn"]

In [ ]:
def get_restaurants(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    rest_name = [tag.text.strip() for tag in soup.find_all('a', class_="restnt-name ellipsis")]
   
    cuisine_spans = soup.find_all('span', class_="double-line-ellipsis")
    cuisine = [ span.text.split('|')[1].strip() if len(span.text.split('|')) > 1 else '' for span in cuisine_spans]
   
    location = [div.text for div in soup.find_all('div', class_="restnt-loc ellipsis")]
    img_url = [img.get('data-src') for img in soup.find_all('img', class_="no-img")]
    rating = [div.text for div in soup.find_all('div', class_="restnt-rating rating-4")]
    
    return pd.DataFrame({'Restaurant name': rest_name ,
                         'Cuisine': cuisine, 
                         'Location': location, 
                         'Ratings': rating, 
                         'Image URL': img_url })

url = "https://www.dineout.co.in/delhi-restaurants/buffet-special"
get_restaurants(url)

In [211]:
xpath_link = '//label[@for="Shoes"]'
tryc:
    link = driver.find_element(By.XPATH, xpath_link)
    link.click()
    print('Shoes selected...')
except NoSuchElementException as e:
    print(f"The link can not be found: {e.msg}")

Shoes selected...


In [ ]:
try:
#             actions.move_by_offset(100, 100).perform()

            link = driver.find_element(By.XPATH, xpath_link)
            actions.move_to_element(link).perform()
            
            link.click()
            print(config_path['msg'][i])
        except NoSuchElementException as e:
            print(f"The link can not be found: {e.msg}")
        except ElementClickInterceptedException as e:
            print(f"{e.msg}")    
        except Exception as e:
            print(f"{e.msg}")

### Custom Functions

In [3]:
def get_attribute_el(webelement, attribute):
    '''Wildcard function to retrieve an attribute other than text from a web-element.
    '''
    return (webelement.get_attribute(attribute))

def text_el(webelement):
    return webelement.text

# Defining lambda functions to retrieved element
fn_href = lambda web: get_attribute_el(web,'href')
fn_text = lambda web: text_el(web)

In [4]:
def remove_popup(xpath):
    ''' Checks for an element corresponding to <xpath> and clicks it. 
        Also, prints the corresponding message according to the result.
        '''
    # Check if an element on the new page is present
    try:
        close_button  = driver.find_element(By.XPATH, xpath)
        close_button.click() 
        print("Pop-up removed")
    except NoSuchElementException:
        print("No pop-up windows was found")
    except ElementNotInteractableException as e:
        print(f"Pop-up windows not Interactable: {e.msg}")

In [5]:
def n_shoes_to_scrap(xpath, pattern = r'(\d+)' ):
    ''' Retrieve the total number of shoes in the category ''' 
    reg = driver.find_element(By.XPATH, xpath)

    # Remove the parentheses
    # pattern = r'\((\d+)\)' 
    match = re.search(pattern, reg.text)
    
    if match:
        nreg = int(match.group(1))
    else:
        nreg = 0

    return nreg

In [6]:
def follow_path_to(config_path):
    ''' Follows every xpath sended in path to retrieve the data.
    Atributes
    ---------
    path: Dict with properties 'xpaths' and 'msg'. With the configurations to follow.
    '''
    actions = ActionChains(driver)
    # Path towards the data 
    for i, xpath_link in enumerate(config_path['xpaths']):
        
        # Waiting for the data
        if 'wait_time' in config_path:
            if len(config_path['wait_time']) > 1:
                wait_time = config_path['wait_time'][i]
            else:
                wait_time = config_path['wait_time'][0]
        else:
            wait_time = 5
            
        print(f"Wait time ** {wait_time} **")
        driver.implicitly_wait(wait_time)

        try:
#             actions.move_by_offset(100, 100).perform()

            link = driver.find_element(By.XPATH, xpath_link)
            actions.move_to_element(link).perform()
            
            link.click()
            print(config_path['msg'][i])
        except NoSuchElementException as e:
            print(f"The link can not be found: {e.msg}")
        except ElementClickInterceptedException as e:
            print(f"{e.msg}")    
        except Exception as e:
            print(f"{e.msg}")
        

In [7]:
def scroll_to_recovered_shoes(n_shoes, xpath):
    '''Scroll the page until almost <n_shoes> will be recovered.
    PARAMS
    -------
    n_shoes: int, number of shoes to recover in the page
    xpath: string, XPath to recover the records
    
    RETURN
    -------
    recovered_shoes: int, total of shoes recovered
    n_shoes_: int, number of shoes to recovered (reference)
    '''
    # Retrieve the total shoes in the page
    tags = driver.find_elements(By.XPATH, xpath )  # scraping available shoes
    recovered_shoes = len(tags)
    print(f"Recovered shoes: {recovered_shoes}")
    j = 1

    # Scrolling to load all shoes in the category
    wait = WebDriverWait(driver,15)
    total = (n_shoes // 10) * 10

    while( total > recovered_shoes ):

        print(f"Scrolling {j} times...")
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(10)

        tags = driver.find_elements(By.XPATH, xpath )  # scraping available shoes
        recovered_shoes = len(tags)
        print(f"Recovered shoes: {recovered_shoes}, still to recovered: {n_shoes - recovered_shoes}")
        j = j + 1

    return recovered_shoes, n_shoes

In [8]:
def load_items_by_xpath(driver, n=None, **kwargs):
    ''' Load items on the page with the attributes sent in kwargs.
        n: number of products required
        kwargs: dictionary with key, value = label, XPATH to retrieve 
        Return: a DataFrame with the info of the products
    '''
    data_retrieved = {}
    lenght_ref = -1
    
    for key, value in kwargs.items():
        
        tags = driver.find_elements(By.XPATH, value[0] )  # scraping

        if len(value)>1:
            items = [ value[1](tag) for tag in tags] # If lambda is specified it applies
        else:
            items = [ tag.text for tag in tags]  # Default: text function
            
        if(lenght_ref <0):
            lenght_ref = len(items)
        
        if(lenght_ref == len(items)):
            data_retrieved[key] = items
            print(f"{key} retrieved and included:", len(items))
        else:
            data_retrieved[key] = [None for _ in range(lenght_ref)]
            print(f"{key} retrieved NOT COMPATIBLE:", len(items))

    df = pd.DataFrame(data_retrieved)
        
    if ( bool(n) and (n < len(df))):
        return (df.iloc[0:n,:])
    
    return df

In [9]:
def scrape_items_by_group(driver, xpath_grp, **kwargs):
    '''Retrieves the data by group, in case any attribute is not 
       defined it assigns the not_found_value for that specific item. For this purpouse we use Selenium.  
       
       ATRIBUTES
       ---------
       xpath_grp : string, XPATH to retrieve the group
       kwargs : dict, attributes to find in each group the corresponding element. 
                Since we use BeautifulSoup we need to specify: the tag name, the particular 
                attributes, the function to retrieve the data, and the not_found_value.
       
               e.g.'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', '-']
               
       RETURN
       -------
       a data frame with the data recovered.
       '''
    # Scrapes products group-tags using Selenium
    try:
        elements = driver.find_elements(By.XPATH, xpath_grp)
        print(f"Recovered group-tags: {len(elements)}")
    except NoSuchElementException as e:
        print(f"No elements corresponding to {xpath_grp}: {e.msg}")

    # Convert the elements to HTML strings
    html_strings = [el.get_attribute("outerHTML") for el in elements]
    print(f"Records converted to HTML strings: {len(html_strings)}")  

    # Scraping data using BeautifulSoup for better performance

    # Initialization of the result dataframe
    df = pd.DataFrame()

    for html_string in html_strings:
        soup = BeautifulSoup(html_string, 'html.parser')

        # Initialization of the new row
        row = {}
        for key, config in kwargs.items():
            tag, attr_dict, prop, not_found_value = config
            # Recover element corresponding to attributes defining in kwargs
            element = soup.find(tag, attr_dict)
            if prop is not None:
                row[key] = (element.get(prop) if element!=None else not_found_value)
            else:
                row[key] = (element.text.strip() if element!=None else not_found_value)

        df = df.append(row, ignore_index=True)

    return df

In [10]:
def bs_scrape_item_detail(html_string, **kwargs):
    ''' Scrapes the data specified in the Kwargs arguments using Beautiful Soup.
        ATRIBUTES
       ---------
       html_string : string, html string
       kwargs : dict, attributes to find in each group the corresponding element. 
                Since we use BeautifulSoup we need to specify: the tag name, the particular 
                attributes, the function to retrieve the data, and the not_found_value.

               e.g.'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', '-']
                          [tag, attrs, condition, not_found_value]
       RETURN
       -------
       a dict with the data recovered.
    '''
    soup = BeautifulSoup(html_string, 'html.parser')

    # Initialization of the new row
    row = {}
    for key, config in kwargs.items():
        tag, attrs, condition, not_found_value = config

        # Recover element corresponding to attributes defining in kwargs
        elements = soup.find_all(tag, attrs)

        if condition is not None:
            if callable(condition): # If a condition is provided, apply it to filter elements
                row[key] = [el.text.strip() for el in elements if condition(el.text.strip())]
            else:  # If a prop is provided
                row[key] = [el.get(condition) if el is not None else not_found_value for el in elements]
        else: # Return text value by default
            row[key] = [el.text.strip() if el is not None else not_found_value for el in elements]
    return row

In [11]:
def load_items_by_page(driver, n=None, **kwargs):
    ''' Load the items on the page base on a configuration dictionary <<kwargs>> .
        n: number of products required
        Return: a DataFrame with the info of the items
    '''
    # scraping each general attribute
    if 'ATTR' in kwargs:
        attributes = kwargs['ATTR']
    else:
        attributes = kwargs
    
    df = load_items_by_xpath(driver, None, **attributes) 
    length_ref = df.shape[0]
    
    if 'GROUP' in kwargs:
        xpath_grp = kwargs['GROUP']['XPATH_GRP'][0]
        attributes_grp =  kwargs['GROUP']['ATTR_GRP']
        
        df2 = load_data_by_group(driver, xpath_grp, length_ref,'-', **attributes_grp)
        df_result = pd.concat([df,df2], axis=1)

    if (bool(n) and length_ref > n):
        return df_result[:n] 

    return df_result

In [12]:
def load_data_by_group(driver, xpath_grp, length_ref, not_found_value='-', **kwargs):
    '''Retrieves the data by group, in case any attribute is not 
       defined it returns None for that specific item.  
       
       ATRIBUTES
       ---------
       xpath_grp = XPATH to retrieve the group
       not_found_value = 
       kwargs = attributes to find in each group, with corresponding xpath config
           e.g.'ratings': [By.CLASS_NAME, 'number', lambda wel: text_el(wel)]
       
       RETURN
       ---------
       Return: a data frame with the data recovered'''
    data_retrieved = {}
    # Scraping the group of elements
    tags = driver.find_elements(By.XPATH, xpath_grp)

    # Scraping all the atributes in each group
    for tag_name, xpath_config in kwargs.items():
        items = []

        # Example of xpath_config: 'ratings': [By.CLASS_NAME, 'number', lambda wel: text_el(wel)]
        by, xpath_param, fn = xpath_config

        for i, tag in enumerate(tags):
            try:
                tag_item = tag.find_elements(by, xpath_param )
                if len(tag_item) > 1:
                    # adding attributes as a list
                    items.append([fn(tag) for tag in tag_item])
                elif len(tag_item) > 0:
                    # adding attribute individually
                    items.append(fn(tag_item[0]))
                else:
                    # If there is not attribute '-' : not_found_param
                    items.append(not_found_value)

            except NoSuchElementException:
                print("Element not found")
                items.append(not_found_value)

        if ( not(length_ref) or length_ref == len(items)):
            data_retrieved[tag_name] = items
            print(f"GRP: {tag_name} retrieved and included: {len(items)}" )

    df = pd.DataFrame(data_retrieved)
    return df